In [1]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from __future__ import print_function
import torch

In [3]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/luc/TRAVAIL/DeepGED/MAO/dataset.ds')

G1 = Gs[0]
G2 = Gs[2]
print(y[13],y[23])
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')
plt.figure()

nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

0.0 1.0


In [4]:
#costs={'nodeSubs':1.0,'nodeIns':1.0,'nodeDel':1.0,'edgeSubs':0,'edgeIns':1.0,'edgeDel':1.0}
def construct_cost_matrix(G1,G2,cost):
    n = G1.order()
    m = G2.order()
    
    #C2=torch.zeros(((n+1)*(m+1),(n+1)*(m+1)))    
    C2=cost[3]*torch.tensor(np.array([ [  k!=l and G1.has_edge(k//(m+1),l//(m+1))^G2.has_edge(k%(m+1),l%(m+1)) for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]))        

    D=torch.zeros((n+1)*(m+1))
    D[n*(m+1):]=costs[1]
    D[n*(m+1)+m]=0
    D[np.array([i*(m+1)+m for i in range(n)])]=costs[1]
    D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=costs[0]*torch.tensor([(G1.nodes[k//(m+1)]!=G2.nodes[k%(m+1)]) for k in range(n*(m+1)) if k%(m+1) != m] )

    C2[range(len(C2)),range(len(C2))]=D
    
    
    #C2=C2.masked_fill(mask,costs['edgeDel'])
    
    return C2
x=torch.rand(4,requires_grad=True) # 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
x=torch.exp(x)
costs=x/x.sum()
print(costs)
C=construct_cost_matrix(G1,G2,costs)
with torch.no_grad():
    C2=C.detach().numpy()
    plt.figure(1)
    plt.imshow(C2)
    plt.colorbar()

tensor([0.3214, 0.1487, 0.3421, 0.1878], grad_fn=<DivBackward0>)


In [5]:
G1=Gs[0]
G2=Gs[0]
n=G1.order()
m=G2.order()
C=construct_cost_matrix(G1,G2,costs)
M=(C.max()*torch.ones(((n+1)*(m+1),(n+1)*(m+1))) -C)
M2=M.detach().numpy()
plt.imshow(M2)
plt.colorbar()

import svd
c=torch.diag(C)
D=C-torch.eye(C.shape[0])*c
x0=svd.eps_assigment_from_mapping(torch.exp(-0.5*c.view(n+1,m+1)),10).view((n+1)*(m+1),1) # a améliorer.
x=svd.franck_wolfe(x0,D,c,2,15,n,m)


print('ged=',.5*x.T@D@x+c.T@x)
x0=x0.view(n+1,m+1)
x=x.view(n+1,m+1)
print('x0=',x0)
print('c=',torch.exp(-0.5*c.view(n+1,m+1)))
print('x=',x)
#print(svd.eps_assigment_from_mapping(torch.exp(x.view(n+1,m+1)),50))
#print((x.T@D+c).view(n+1,m+1))

ged= tensor([[6.9162]], grad_fn=<AddBackward0>)
x0= tensor([[0.0778, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0778, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0778, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0662, 0.0778, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0662, 0.0662, 0.0778, 0.0662, 0.0662, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0778, 0.0662, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0778, 0.0662, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0662, 0.0778, 0.0662,
         0.0662, 0.0662, 0.2580],
        [0.0662, 0.0662, 0.0

In [6]:
#u, l, v = torch.svd(M)
e,v=torch.symeig(M,eigenvectors=True)
#print(e[-1])
#print('w=',w)
eig_val=e[-1]
#print('w[-1]=',w[:,-1])

first_ev=v[:,-1]
if(first_ev.sum() <0):
    first_ev=-first_ev
print(eig_val.item())
print('first_ev=',first_ev)
print((first_ev@M@first_ev.t()))

38.58636474609375
first_ev= tensor([0.0842, 0.0799, 0.0835, 0.0801, 0.0834, 0.0832, 0.0832, 0.0834, 0.0801,
        0.0832, 0.0865, 0.0902, 0.0799, 0.0780, 0.0799, 0.0774, 0.0798, 0.0797,
        0.0797, 0.0798, 0.0774, 0.0797, 0.0821, 0.0849, 0.0835, 0.0799, 0.0842,
        0.0801, 0.0834, 0.0832, 0.0832, 0.0834, 0.0801, 0.0832, 0.0865, 0.0902,
        0.0801, 0.0774, 0.0801, 0.0782, 0.0800, 0.0799, 0.0799, 0.0800, 0.0775,
        0.0799, 0.0823, 0.0851, 0.0834, 0.0798, 0.0834, 0.0800, 0.0839, 0.0831,
        0.0831, 0.0832, 0.0800, 0.0831, 0.0863, 0.0900, 0.0832, 0.0797, 0.0832,
        0.0799, 0.0831, 0.0837, 0.0830, 0.0831, 0.0799, 0.0830, 0.0862, 0.0898,
        0.0832, 0.0797, 0.0832, 0.0799, 0.0831, 0.0830, 0.0837, 0.0831, 0.0799,
        0.0830, 0.0862, 0.0898, 0.0834, 0.0798, 0.0834, 0.0800, 0.0832, 0.0831,
        0.0831, 0.0839, 0.0800, 0.0831, 0.0863, 0.0900, 0.0801, 0.0774, 0.0801,
        0.0775, 0.0800, 0.0799, 0.0799, 0.0800, 0.0782, 0.0799, 0.0823, 0.0851,
        0.08

In [7]:
S0 = first_ev.view(G1.order()+1,G2.order()+1)
Splot=S0.detach().numpy()
plt.imshow(Splot)
#print(np.sum(S0,axis=0))
#print(np.sum(S0,axis=1))
plt.colorbar()

print(S0)

tensor([[0.0842, 0.0799, 0.0835, 0.0801, 0.0834, 0.0832, 0.0832, 0.0834, 0.0801,
         0.0832, 0.0865, 0.0902],
        [0.0799, 0.0780, 0.0799, 0.0774, 0.0798, 0.0797, 0.0797, 0.0798, 0.0774,
         0.0797, 0.0821, 0.0849],
        [0.0835, 0.0799, 0.0842, 0.0801, 0.0834, 0.0832, 0.0832, 0.0834, 0.0801,
         0.0832, 0.0865, 0.0902],
        [0.0801, 0.0774, 0.0801, 0.0782, 0.0800, 0.0799, 0.0799, 0.0800, 0.0775,
         0.0799, 0.0823, 0.0851],
        [0.0834, 0.0798, 0.0834, 0.0800, 0.0839, 0.0831, 0.0831, 0.0832, 0.0800,
         0.0831, 0.0863, 0.0900],
        [0.0832, 0.0797, 0.0832, 0.0799, 0.0831, 0.0837, 0.0830, 0.0831, 0.0799,
         0.0830, 0.0862, 0.0898],
        [0.0832, 0.0797, 0.0832, 0.0799, 0.0831, 0.0830, 0.0837, 0.0831, 0.0799,
         0.0830, 0.0862, 0.0898],
        [0.0834, 0.0798, 0.0834, 0.0800, 0.0832, 0.0831, 0.0831, 0.0839, 0.0800,
         0.0831, 0.0863, 0.0900],
        [0.0801, 0.0774, 0.0801, 0.0775, 0.0800, 0.0799, 0.0799, 0.0800, 0.0782,

In [8]:
def proj_bisto_alter_eps_assig(S0):
    ones_n = torch.ones(S0.shape[0])
    ones_m = torch.ones(S0.shape[1])
    print("ones_n:",ones_n)
    Sk = S0
    for i in range(20):
        D=torch.diag(1.0/(torch.matmul(Sk,ones_m)))
        D[D.shape[0]-1,D.shape[1]-1]=1.0
        Sk1 = torch.matmul(D,Sk)
        D=torch.diag(1.0/(torch.matmul(ones_n,Sk1)))
        D[D.shape[0]-1,D.shape[1]-1]=1.0
        Sk = torch.matmul(Sk1,D)
        
    return Sk

S0=torch.exp(S0)

S = proj_bisto_alter_eps_assig(S0)
print("lines:",torch.sum(S,0))
print("columns:",torch.sum(S,1))
Splot=S.detach().numpy()[0:(S.shape[0]-1),0:(S.shape[1]-1)]
plt.imshow(Splot)
plt.colorbar()
v=torch.flatten(S)
print(S)
#print(v)
ged=.5*(v.t()@C@v)
print('GED=', ged.item())
out=(y[13]!=y[23])*ged-(y[13]==y[23])*ged
print(out)
out.backward()
#torch.nn.HingeEmbeddingLoss(margin=1.0, size_average=None, reduce=None, reduction='mean')

ones_n: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
lines: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 4.0776], grad_fn=<SumBackward1>)
columns: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 4.0776], grad_fn=<SumBackward1>)
tensor([[0.0664, 0.0663, 0.0663, 0.0663, 0.0663, 0.0663, 0.0663, 0.0663, 0.0663,
         0.0663, 0.0663, 0.2703],
        [0.0663, 0.0665, 0.0663, 0.0664, 0.0663, 0.0664, 0.0664, 0.0663, 0.0664,
         0.0664, 0.0663, 0.2699],
        [0.0663, 0.0663, 0.0664, 0.0663, 0.0663, 0.0663, 0.0663, 0.0663, 0.0663,
         0.0663, 0.0663, 0.2703],
        [0.0663, 0.0664, 0.0663, 0.0665, 0.0663, 0.0664, 0.0664, 0.0663, 0.0664,
         0.0664, 0.0663, 0.2700],
        [0.0663, 0.0663, 0.0663, 0.0663, 0.0664, 0.0663, 0.0663, 0.0663, 0.0663,
         0.0663, 0.0663, 0.2703],
        [0.0663, 0.0664, 0.0663, 0.0664, 0.0663, 0.0664, 0.0663, 0.066